In [30]:
import pandas as pd
import numpy as np
import scipy.special
from scipy.stats import chi2

In [2]:
data = pd.DataFrame(
    {
        "st zlomov": pd.Series(range(6)),
        "st palic": pd.Series([157, 69, 35, 17, 1, 1])
    }
)

In [3]:
data


,st zlomov,st palic
0,0,157
1,1,69
2,2,35
3,3,17
4,4,1
5,5,1


In [7]:
n = sum(data['st palic'])
X_bar = sum(data['st palic']*data['st zlomov']) / n

p_hat = X_bar/5

In [22]:
def bin_distrib(x, p, n=5):
    return scipy.special.comb(n, x, exact=True) * p**x * (1 - p)**(n - x)

In [23]:
E = pd.Series([n * bin_distrib(x, p_hat, 5) for x in range(6)])

X2 = sum(((data['st palic'] - E)**2) / E)

In [29]:
sum((data['st palic'][i] - E[i])**2 / E[i] for i in range(6)) 

100.3797880290121

In [35]:
alpha = 0.01
chi2.ppf(1 - alpha, 5)
# testna statistika X2 je na našem vzorcu veliko večja, 
# zato ničelno hipotezo zavržemo tako pri stopnji tveganja 5% kot tudi 1%.

15.08627246938899

In [52]:
Lambda = np.prod([((X_bar**x)/(x**x) * (5 - X_bar)**(5 - x)/(5 - x)**(5 - x))**data['st palic'][x] for x in range(6)])

np.log(Lambda)

-222.24428374568322

In [51]:
(X_bar**0 * (5 - X_bar)**5/(5**5))**157

5.37963491842219e-53

In [54]:
scipy.special.comb(10, 5) * (1/2)**10

0.24609375